In [1]:
import os 
import csv
import pandas as pd
import numpy as np
import streamlit as st
import random
import torch
from transformers import TapasTokenizer, TapasForQuestionAnswering

def load_model():
    print('downloading model')
    model_name = 'google/tapas-base-finetuned-wtq'
    model = TapasForQuestionAnswering.from_pretrained(model_name)
    tokenizer = TapasTokenizer.from_pretrained(model_name)
    print('model downloaded')
    return model,tokenizer

/home/ec2-user/streamlit-tapas/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
from transformers.utils import logging
logging.set_verbosity_error() 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
model, tokenizer = load_model()

downloading model


/home/ec2-user/streamlit-tapas/venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model downloaded


In [3]:
import pandas as pd
df = pd.read_csv('../data/happiness.csv')

In [11]:
def map_answer(inputs,outputs,data):
    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions( inputs, outputs.logits.detach(), outputs.logits_aggregation.detach())   
    id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3:"COUNT"}
    aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

    answers = []
    
    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            # only a single cell:
            answers.append(data.iat[coordinates[0]])
        else:
            # multiple cells
            cell_values = []
            for coordinate in coordinates:
                cell_values.append(data.iat[coordinate])
            print(cell_values)
            answers.append(", ".join(cell_values))
    return answers

In [17]:
input_queries = ['What is the ratio of male to female in this dataset?',
                 'Which region has the highest average income in this dataset?']
inputs = tokenizer(table=df.head(100).astype(str), queries=input_queries, padding='max_length', return_tensors="pt",truncation=True)
outputs = model(**inputs)
map_answer(inputs,outputs,df)


[46.0, 'middle atlantic']

In [1]:
# map_answer(inputs,outputs,df)